In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras

- Importation des données

In [2]:
df = pd.read_csv("data.csv")
print(df.head())

       timestamp  temperature  humidity  mq2_gas  mq135_air_quality
0  1742857322661         24.0      40.0     3628                  0
1  1742857332981         24.0      40.0     3628                  0
2  1742857342902         24.0      40.0     3628                  0
3  1742857352289         24.0      40.0     3628                  0
4  1742857362285         24.0      40.0     3628                  0


In [3]:
df.describe()

,timestamp,temperature,humidity,mq2_gas,mq135_air_quality
count,4.340000e+02,434.000000,434.000000,434.000000,434.000000
mean,1.742985e+12,18.409631,47.340507,2795.442396,3946.175115
std,1.021793e+08,8.591340,9.198330,1298.026143,13740.414239
min,1.742857e+12,-4.900000,36.000000,873.000000,0.000000
25%,1.742860e+12,10.110000,40.000000,1545.000000,0.000000
50%,1.743035e+12,23.690000,45.000000,3578.000000,24.000000
75%,1.743096e+12,24.000000,52.500000,3628.000000,1764.250000
max,1.743099e+12,59.600000,95.000000,5588.000000,65416.000000


In [4]:
df.isnull().mean()*100

timestamp            0.0
temperature          0.0
humidity             0.0
mq2_gas              0.0
mq135_air_quality    0.0
dtype: float64

In [5]:
df.duplicated().mean()*100

0.0

- timestamp est inutile pour l'entrainement, on le retire

In [6]:
df.drop(columns=['timestamp'], inplace=True)

- X et y

In [11]:
X = df[['temperature', 'humidity']]
y = df[['mq2_gas', 'mq135_air_quality']] 

- On normalise nos données

In [12]:
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)
y_norm= scaler.fit_transform(y)

- diviser les ensembles

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y_norm, test_size=0.2, random_state=42)

- le modèle

In [16]:
model = keras.Sequential([
    keras.layers.Dense(8, activation='relu', input_shape=(2,)),  
    keras.layers.Dense(4, activation='relu'),  
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(2, activation='sigmoid')
])
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])

- Entrainement du modèle

In [17]:
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9646 - loss: 0.1476 - val_accuracy: 0.9714 - val_loss: 0.1325
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9319 - loss: 0.1334 - val_accuracy: 0.9714 - val_loss: 0.1189
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9396 - loss: 0.1215 - val_accuracy: 0.9714 - val_loss: 0.1060
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9500 - loss: 0.1073 - val_accuracy: 0.9714 - val_loss: 0.0939
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9512 - loss: 0.0925 - val_accuracy: 0.9714 - val_loss: 0.0831
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9498 - loss: 0.0860 - val_accuracy: 0.9714 - val_loss: 0.0739
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9342 - loss: 0.0816 - val_accuracy: 0.9714 - val_loss: 0.0663
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9382 - loss: 0.0734 - val_accuracy: 0.9714 - val_l

In [19]:
predictions = model.predict(X_test)
mq2_residuals = abs(y_test[:, 0] - predictions[:, 0])
mq135_residuals = abs(y_test[:, 1] - predictions[:, 1])

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


In [26]:
mq2_threshold = np.percentile(mq2_residuals, 95)
mq135_threshold = np.percentile(mq135_residuals, 95)
mq135_threshold

0.6645716852652583

-sauvegarde du modèle

In [ ]:
model.save("anomaly_detector.keras")
print("Modèle keras entraîné et sauvegardé.")

Modèle keras entraîné et sauvegardé.


-conversion en tflite

In [22]:
# Conversion en modèle TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Sauvegarde du modèle TFLite
with open("anomaly_detector.tflite", "wb") as f:
    f.write(tflite_model)

print("Modèle converti en TensorFlow Lite et sauvegardé.")

INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpa4j2bcl1\assets


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpa4j2bcl1\assets


Saved artifact at 'C:\Users\hp\AppData\Local\Temp\tmpa4j2bcl1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  1613412557520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1613412558096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1613412557904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1613412559056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1613412558864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1613412559632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1613412556368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1613412560976: TensorSpec(shape=(), dtype=tf.resource, name=None)
Modèle converti en TensorFlow Lite et sauvegardé.


- Convertir en tableau c++ pour pouvoir intégrer à wokwii

In [23]:
# Charger le modèle TFLite
with open("anomaly_detector.tflite", "rb") as f:
    tflite_model = f.read()

# Convertir en tableau C++
hex_array = ', '.join(f'0x{b:02x}' for b in tflite_model)

# Générer le fichier C++
cpp_code = f"""
#include <cstddef>

alignas(8) const unsigned char anomaly_model_tflite[] = {{
    {hex_array}
}};

const size_t anomaly_model_tflite_len = {len(tflite_model)};
"""

# Sauvegarder en fichier .h
with open("anomaly_model.h", "w") as f:
    f.write(cpp_code)

print("Fichier anomaly_model.h généré avec succès !")

Fichier anomaly_model.h généré avec succès !


In [24]:
import os

model_path = "anomaly_detector.tflite"
model_size = os.path.getsize(model_path)

print(f"Taille du modèle : {model_size} octets")

Taille du modèle : 2924 octets
